# COVID analysis based on the number of deceased cases

* https://medicalxpress.com/news/2020-04-covid-average-actual-infections-worldwide.html
* https://www.thelancet.com/pdfs/journals/laninf/PIIS1473-3099(20)30243-7.pdf

In [20]:
!pwd
!curl -O https://data.covid19japan.com/summary/latest.json

/Users/hide/_projects/jupyter_notebooks/COVID-19/COVID-modeling
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  104k  100  104k    0     0   121k      0 --:--:-- --:--:-- --:--:--  121k


In [21]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from ipywidgets import interact

%matplotlib inline

In [22]:
def load_data(filepath):
    with open(filepath, 'r') as f:
        data = json.loads(f.read())
    df = pd.DataFrame(data['daily'])
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    columns = [col for col in df.columns if 'Avg' not in col]
    df = df[:-1][columns]  # cut off the last row
    return df

def get_daily(df, column_name):
    return df[column_name] - df[column_name].shift(1)

def append_column(input_df):
    df = input_df.copy()
    df['exitedCumulative'] = df['recoveredCumulative'] + df['deceasedCumulative']
    
    df['dailyTested'] = get_daily(df, 'testedCumulative')
    df['dailyConfirmed'] = get_daily(df, 'confirmedCumulative')
    df['dailyDeceased'] = get_daily(df, 'deceasedCumulative')
    
    df['dailyConfirmedInTestedRate'] = df['dailyConfirmed'] / df['dailyTested']
    return df
    
df = append_column(load_data('latest.json'))
df.tail(3).transpose()

date,2020-04-13,2020-04-14,2020-04-15
confirmed,276.000000,476.000000,521.000000
recoveredCumulative,799.000000,853.000000,901.000000
deceasedCumulative,129.000000,139.000000,178.000000
criticalCumulative,135.000000,152.000000,168.000000
testedCumulative,78702.000000,89551.000000,94236.000000
confirmedCumulative,7726.000000,8202.000000,8723.000000
exitedCumulative,928.000000,992.000000,1079.000000
dailyTested,1321.000000,10849.000000,4685.000000
dailyConfirmed,276.000000,476.000000,521.000000
dailyDeceased,4.000000,10.000000,39.000000


In [23]:
# Check testing rate
remove_anomaly_df = df[df['dailyConfirmedInTestedRate'] < 1]

@interact(n=(1, 30))
def plot_rolling_mean(n=7):
    fig, ax = plt.subplots(1, 2, figsize=(20, 8))

    df[['dailyTested']].plot(ax=ax[0], title=f'Daily Tested (Moving average: {n} days)')
    df[['dailyTested']].rolling(n).mean().plot(ax=ax[0], grid=True)
    remove_anomaly_df[['dailyConfirmedInTestedRate']].plot(ax=ax[1], title=f'Daily Confirmed/Tested Rate (Moving average: {n} days)')
    remove_anomaly_df[['dailyConfirmedInTestedRate']].rolling(n).mean().plot(ax=ax[1], grid=True)

interactive(children=(IntSlider(value=7, description='n', max=30, min=1), Output()), _dom_classes=('widget-int…

In [24]:
# Perform deceased base analysis
from ipywidgets import interact

DEFAULT_FATALITY_RATE = 1.6
DEFAULT_DETECTION_RATE = 25
DEFAULT_DAYS_FROM_TESTED_TO_DEATH = 14

def append_estimated_column(input_df, fatality_rate, detection_rate, days_from_tested_to_death):
    # Decease base estimated confirmed
    df = input_df.copy()

    df['estimatedConfirmedCumulative'] = df['deceasedCumulative'].shift(-days_from_tested_to_death) / fatality_rate
    df['detectionRate'] = df['confirmedCumulative'] / df['estimatedConfirmedCumulative']
    df['estimatedConfirmedCumulativeByDetectionRate'] = df['confirmedCumulative'] / detection_rate
    
    return df

@interact(fatality_rate_pct=(1.0, 5.0), detection_rate_pct=(10, 50), days_from_tested_to_death=(8, 30))
def plot_estimated_data(
    fatality_rate_pct=DEFAULT_FATALITY_RATE,
    detection_rate_pct=DEFAULT_DETECTION_RATE,
    days_from_tested_to_death=DEFAULT_DAYS_FROM_TESTED_TO_DEATH

):
    new_df = append_estimated_column(df, fatality_rate_pct/100, detection_rate_pct/100, days_from_tested_to_death)
    
    fig, ax = plt.subplots(1, 3, figsize=(24, 8))

    new_df[[
        'estimatedConfirmedCumulative',
        'deceasedCumulative',
    ]].plot(ax=ax[0], grid=True, title=f'Estiamted Infected cases calculated from Decease cases\n* Days from Tested to Deceased ={days_from_tested_to_death} days \n* Fatality rate = {fatality_rate_pct}%')

    new_df[[
        'confirmedCumulative',
        'estimatedConfirmedCumulative',
        'estimatedConfirmedCumulativeByDetectionRate',
    ]].plot(ax=ax[1], grid=True, title=f'Estiamted Infected cases calculated by Estimated Detection Rate\n(detection rate = {detection_rate_pct}%)')

    new_df[['detectionRate']].plot(ax=ax[2], grid=True, title='Detection Rate from Estimated Infected cases')

interactive(children=(FloatSlider(value=1.6, description='fatality_rate_pct', max=5.0, min=1.0), IntSlider(val…

# Fit the estimated number to SIR model

In [ ]:
import json
import pandas as pd

from ddeint import ddeint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from scipy.integrate import odeint

import optuna
from sklearn.metrics import mean_squared_error, mean_squared_log_error

In [ ]:
# Initial values
N = 126_100_000
I0 = 62
R0 = 0
S0 = N - I0 - R0
init_state = [S0, I0, R0]

def model(Y, t, N, beta, gamma):
    '''SIR Model'''
    S = Y[0]
    I = Y[1]
    R = Y[2]
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return np.array([
        dSdt, dIdt, dRdt
    ])

def create_data_range_and_tt(start_date, end_date):
    date_range = pd.date_range(start_date, end_date, freq='d')
    t_max = len(date_range)
    tt = np.linspace(0.0, t_max-1, t_max)
    return date_range, tt

def objective_with_args(start_date, end_date):
    date_range, tt = create_data_range_and_tt(start_date, end_date)

    def objective(trial):
        beta = trial.suggest_loguniform('beta', 0.001, 0.5)
        gamma = trial.suggest_loguniform('gamma', 0.03, 0.2)
        yy = odeint(model, init_state, tt, args=(N, beta, gamma))
        return (
            mean_squared_error(yy[:,1], df.loc[start_date:end_date]['estimatedConfirmedCumulativeByDetectionRate'])
        )

    return objective

In [ ]:
# Date for training
start_date = '2020-02-10'
end_date = '2020-04-11'

optuna.logging.disable_default_handler()
study = optuna.create_study()
study.optimize(objective_with_args(start_date, end_date), n_trials=100)
print("best_value = ", study.best_value)
print("best_params = ", study.best_params)

# Initial values
beta = study.best_params['beta']
gamma = study.best_params['gamma']

print(f'infection rate = {beta}')
print(f'days = {1/gamma}')
print(f'R0 = {beta/gamma}')

In [ ]:
start_date = '2020-02-10'
end_date = '2020-04-16'
date_range, tt = create_data_range_and_tt(start_date, end_date)

new_yy = odeint(model, init_state, tt, args=(N, beta, gamma))

def make_df(yy, date_range):
    df = pd.DataFrame(yy[:, 0:3], columns=['pred_S', 'pred_I', 'pred_R'])
    df.index = date_range
    df['pred_D'] = df['pred_I'].shift(14) * fatality_rate
    df['pred_I2'] = df['pred_I'] * detection_rate
    return df

predicted_df = make_df(new_yy, date_range)
df_with_pred = pd.merge(df, predicted_df, how='outer', left_index=True, right_index=True)
df_with_pred[[
#     'estimatedConfirmedCumulative',
#     'estimatedConfirmedCumulativeByDetectionRate',
#     'pred_I',
    'confirmedCumulative',
    'pred_I2',
#     'testedCumulative'
    ]].plot()

In [ ]:
df_with_pred[[
    'deceasedCumulative',
    'pred_D'
    ]].plot()

In [ ]:
df_with_pred[[
    'confirmedCumulative',
    'pred_I2',
    'testedCumulative'
    ]].plot()